In [24]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction.text import CountVectorizer


In [2]:
business = pd.read_json('yelp_dataset/business.json', lines=True)
#checkin = pd.read_json('yelp_dataset/checkin.json', lines=True)
#photo = pd.read_json('yelp_dataset/photo.json', lines=True)
##review = pd.read_json('yelp_dataset/review.json', lines=True)
#tip = pd.read_json('yelp_dataset/tip.json', lines=True)
##user = pd.read_json('yelp_dataset/user.json', lines=True)

### Only look at restaurants

In [3]:
#lv_restaurants
restaurants = business['categories'].str.contains('Restaurants', regex=False)
restaurants = restaurants.fillna(False)
restaurants = business[restaurants]
print(restaurants.shape)
restaurants.head()

(59371, 14)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,None,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV


In [44]:
restaurants[restaurants['categories'].str.contains('repair', case=False)]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
19330,"1351 W Sunset Rd, Ste 110","{'WheelchairAccessible': 'True', 'DogsAllowed'...",AWWqMTqLx9EYyYHOwZ8yBA,"Local Services, Shopping, Restaurants, Compute...",Henderson,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",0,36.063157,-115.044634,Device Pitstop Henderson,89014,22,3.5,NV
25500,1250 College Street,"{'HasTV': 'False', 'Alcohol': 'u'none'', 'Whee...",4CDBiS3Q1JPk4WicswFPnA,"Coffee & Tea, Local Services, Food, IT Service...",Toronto,"{'Wednesday': '9:30-17:0', 'Thursday': '9:30-1...",1,43.651534,-79.438358,De Floured,M6H 1C2,8,5.0,ON
38056,1366 Old Freeport Rd,"{'RestaurantsPriceRange2': '3', 'RestaurantsGo...",Fbudk6ffw8DlLwKwunilZA,"Restaurants, Boat Dealers, Automotive, Marinas...",Pittsburgh,"{'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'...",1,40.486365,-79.872736,Fox Chapel Yacht Club,15238,5,2.0,PA
38855,516 E Laguna Dr,"{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",UR7_pPtTqN5kjh_QBIGYYQ,"Pizza, Restaurants, Flooring, Foundation Repai...",Tempe,None,1,33.429864,-112.347746,Concrete Repairman,85282,10,2.5,AZ
46104,75 Sun Valley Blvd,"{'RestaurantsPriceRange2': '2', 'HasTV': 'True...",kiicjb7v_8B_TMHjj86SCQ,"Breakfast & Brunch, Restaurants, Auto Repair, ...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,50.906521,-114.047815,NAPA AUTOPRO - MRI AutoCare,T2X 2G6,3,3.5,AB
47322,"2727 Rue Saint-Patrick, Suite 1","{'WheelchairAccessible': 'True', 'GoodForKids'...",XFZwZTABDk09jfv6tEJR6Q,"Hotels & Travel, Bike Rentals, Active Life, Re...",MontrÃ©al,"{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ...",1,45.478696,-73.573600,Ma Bicyclette,H3K 0A8,13,4.5,QC
54432,20 Research Ln,"{'BusinessParking': '{'garage': False, 'street...",y8wucy_3sH2FGE1auFpkXQ,"Kitchen & Bath, Metal Fabricators, Wholesalers...",Las Vegas,"{'Monday': '9:0-16:0', 'Tuesday': '9:0-16:0', ...",1,36.159889,-115.103245,Golden Bar & Restaurant Equipment,89101,4,5.0,NV
61991,"7322 S Rainbow Blvd, Ste 117","{'Ambience': '{'romantic': False, 'intimate': ...",IjsLANGkmAqCsF6-zgIA8w,"Event Planning & Services, Fashion, Food Court...",Las Vegas,"{'Monday': '9:0-23:0', 'Tuesday': '9:0-23:0', ...",1,36.055128,-115.242644,Best Of The Best DJ's,89139,18,4.0,NV
66693,2925 W Whitton Ave,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",mxK5rU9bikniqrHw7D6cEg,"Shopping, Kitchen & Bath, Furniture Stores, Ap...",Phoenix,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",1,33.489659,-112.122017,Economic Equipment Locator,85017,7,5.0,AZ
72526,"2076 E University Dr, Ste A","{'RestaurantsPriceRange2': '1', 'WiFi': ''free...",b2-K2sJtXv7dynBDFqusBQ,"Shopping, Local Services, Cafes, Home & Garden...",Tempe,"{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",1,33.422127,-111.898018,Coyote Coffee Cafe,85281,56,4.0,AZ


In [17]:
X_train, X_test, y_train, y_test = train_test_split(restaurants['categories'], \
                                                    restaurants['stars'], test_size=0.33, random_state=42)


In [ ]:
X_train = pd.Series(X_train).str.get_dummies(sep=', ')
X_test = pd.Series(X_test).str.get_dummies(sep=', ')

In [45]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
categories_count = vectorizer.fit_transform(restaurants['categories']).astype(np.int8)
categories_list = vectorizer.get_feature_names()

In [51]:
categories_df = pd.DataFrame(categories_count.toarray())
categories_df.columns = categories_list

In [52]:
categories_df.head()

,acai,accessories,accountants,acne,active,activities,acupuncture,adoption,adult,advertising,...,wineries,wings,women,wraps,yelp,yoga,yogurt,your,yourself,zoos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Restaurants,Food,Pizza,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Chinese,Restaurants,Food,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Cheesesteaks,Food,Restaurants,American (Traditional),Food Trucks,Sandwiches,Specialty Food,Street Vendors,Cheese Shops,None,...,None,None,None,None,None,None,None,None,None,None
3,Restaurants,Pizza,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Food,Grocery,Seafood Markets,Filipino,Specialty Food,Restaurants,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [24]:
X_test.head()

,& Probates,Acai Bowls,Accessories,Acne Treatment,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,...,Wine & Spirits,Wine Bars,Wine Tasting Classes,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,Zoos
67682,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
56469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#check for columns missing from test set
missing_train = list(set(list(X_train)) - set(list(X_test)))
for col in missing_train:
    X_test[col] = 0
    
missing_test = list(set(list(X_test)) - set(list(X_train)))
for col in missing_test:
    X_train[col] = 0

In [26]:
X_train.head()

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult Education,Adult Entertainment,Advertising,Afghan,African,...,Carpet Installation,& Probates,Traditional Clothing,Dance Schools,Tonkatsu,Gardeners,Trophy Shops,Counseling & Mental Health,Campgrounds,Ice Delivery
68223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X_test.head()

,& Probates,Acai Bowls,Accessories,Acne Treatment,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,...,Holistic Animal Care,Foundation Repair,Guest Houses,Bridal,Parking,Family Practice,Studio Taping,Teeth Whitening,Print Media,Medical Transportation
67682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
print(X_train.shape)
print(X_test.shape)

(39778, 761)
(19593, 761)


In [30]:
y_train_cut = pd.cut(np.array(y_train), 2, labels=[1, 2])
y_test_cut = pd.cut(np.array(y_test), 2, labels=[1, 2])
np.array(y_train_cut)

array([2, 1, 2, ..., 1, 2, 2])

In [31]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, np.array(y_train_cut)) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [32]:
np.array(y_test_cut)

array([2, 2, 2, ..., 1, 1, 2])

In [33]:
neigh.score(X_test, np.array(y_test_cut))

0.5274332669831062